### Virtual Environment
To keep our versions the same across machines, we'll use anaconda's built-in virtual environment manager. Run the following from the commandline. 
<pre>
$conda remove --name activerator_env --all
$conda env create -f environment.yml
$source activate activerator_env
</pre>

In [1]:
from imports import *
import MySQLdb
import json as pp
import mysql.connector

### Configs
First, [build a stava application](https://www.strava.com/settings/api). 

Next, enter the following values in a `config.cfg` file in this repo:
<pre>
[creds]
client_id = enterID
client_secret = enterSecret
access_token = enterToken
</pre>


In [2]:
config = ConfigParser.RawConfigParser()
config.read('config.cfg')
client_id = config.get('creds','client_id')
client_secret = config.get('creds','client_secret')
access_token = config.get('creds','access_token')
header = {
    'Authorization': 'Bearer {}'.format(access_token)
}
rds_db = config.get('rds','DB_instance')
rds_usr = config.get('rds','user_name')
rds_pswd = config.get('rds','pasword')
print rds_db

activerator-rds


### Activerator
We're going to learn how to use the strava v3 api. We'll first build a function that allows us to make GET requests and return a json result. 

In [ ]:
def make_request(p,access_token,endpoint):
    """
    Initiates a get request w/ specific parameters provided.
    """
    return requests.get(
            config.get('creds','url')+endpoint
            , params=p
            , headers = header
        ).json()


Next, let's get our own list of the bikes we loaded in strava using the [athelete endpoint](http://strava.github.io/api/v3/athlete/). 

In [ ]:
endpoint = 'athlete'
parameters = {
    'per_page': 2 #up to 200
}
print pp.dumps(make_request(parameters, access_token, endpoint)['bikes'],indent=4)

We can explore efforts on a specific segment over a specific date range using the [segments efforts endpoint](http://strava.github.io/api/v3/efforts/). 

In [ ]:
!ls

In [ ]:
endpoint = 'segments/636162/all_efforts'
parameters = {
    'per_page': 2 #up to 200
    , 'start_date_local': '2016-02-28T00:00:00Z'
    , 'end_date_local':'2016-02-28T23:59:59Z'
}
#print pp.dumps(make_request(parameters, access_token, endpoint),indent=4)
data = make_request(parameters, access_token, endpoint)
with open('data/initial_segment636162_data.json','wb') as result:
    result.write(json.dumps(data))


In [ ]:
!cat data/initial_segment636162_data.json

The process of installing the MySQLdb library was strange. Documentation is below.

* `$ pip install MySQL-python`

I also had to do the following:
* `pip install jupyter`. 

Then I received this error:
<pre>
ImportError: dlopen(/Users/blehman/anaconda/envs/activerator_env/lib/python2.7/site-packages/_mysql.so, 2): Library not loaded: libssl.1.0.0.dylib
  Referenced from: /Users/blehman/anaconda/envs/activerator_env/lib/python2.7/site-packages/_mysql.so
  Reason: image not found
</pre>

I found this [stackoverflow conversation](http://stackoverflow.com/questions/6383310/python-mysqldb-library-not-loaded-libmysqlclient-18-dylib) and followed [this particular solution](http://stackoverflow.com/questions/6383310/python-mysqldb-library-not-loaded-libmysqlclient-18-dylib?id=answer-6967816#answer-6967816). I had to first find where `libcrypto.1.0.0.dylib` was located and I did that by searching (with `command space`). Once I had the location (ie. `~/anaconda/lib/libcrypto.1.0.0.dylib`)  creates a symbolic link in /usr/lib to the appropriate file:
<pre>
$ sudo ln -s ~/anaconda/lib/libcrypto.1.0.0.dylib /usr/lib/libcrypto.1.0.0.dylib
</pre>



In [ ]:
!conda install mysql-connector-python
!conda install pymysql

Using [mysql-connector-python](https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html#idm140107932776752) with [examples](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html), I can now create a database locally. 
1. Get mysql running locally.
2. `$ mysql`
2. `mysql> GRANT ALL ON testdbinstance.* TO '<user_name>'@'localhost' IDENTIFIED BY <pswd>`

In [4]:
cnx = mysql.connector.connect(user='tester'
                              , password='lY81KA2mA7u9aWgGWu3q'
                              #, host='localhost'
                              #, host='127.0.0.1'
                              , database='testdbinstance'
                              , host='localhost'
                              )#, unix_socket='/tmp/mysql.sock')
cursor = cnx.cursor()

In [ ]:
cnx = mysql.connector.connect(user=rds_
                              , password='lY81KA2mA7u9aWgGWu3q'
                              #, host='localhost'
                              #, host='127.0.0.1'
                              , database='testdbinstance'
                              , host='localhost'
                              )#, unix_socket='/tmp/mysql.sock')
cursor = cnx.cursor()

In [ ]:
!rm /tmp/mysql.sock

In [ ]:
cnx = mysql.connector.connect(user='tester'
                              , password='lY81KA2mA7u9aWgGWu3q'
                              , host='127.0.0.1'
                              , database='testdbinstance'
                              , unix_socket='/tmp/mysql.sock')

In [ ]:
from mysql.connector import (connection)
cnx = connection.MySQLConnection(user=rds_usr
                                 , password=rds_pswd
                                 , host='127.0.0.1'
                                 , database=rds_db
                                 , unix_socket='/tmp/mysql.sock')

In [ ]:
##
# Sarah - could you help with this code below?
##

# establish a connection (currently getting an OperationalError)
conn = MySQLdb.connect(
    #host="127.0.0.1"
    host='localhost'
    , port = 3306
    , user=rds_usr
    , passwd=rds_pswd
    , db=rds_db
    , unix_socket='/tmp/mysql.sock' 
    )

# create table
# insert data
# query data 

In [ ]:
import pymysql.cursors
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user=rds_usr,
                             password=rds_pswd,
                             db=rds_db,
                             charset='utf8mb4',
                             )#cursorclass=pymysql.cursors.DictCursor)


### What's next? 
We hope to compile some data, save it on RDS, and build a D3 visualization that allows the client to explore results.  